In [1]:
from timeit import default_timer as timer
start = timer()

In [2]:
from openpyxl import Workbook
from openpyxl import load_workbook
from numba import vectorize
import numpy as np

In [3]:
wb_query = Workbook()

In [4]:
path = 'D:\EDUCATION\Data Mining\\query_database.xlsx'

In [5]:
wb_query = load_workbook(path)

In [6]:
ws_query = wb_query.active

In [7]:
query_tuple_cells = ws_query['A']

In [8]:
query_labels = []
for cell_num in range(1, len(query_tuple_cells)+1):
    current_cell = 'A'+str(cell_num)
    query_labels.append(ws_query[current_cell].value)

num_query_data = len(query_labels)
print('Total no of query data =', num_query_data)

Total no of query data = 10


In [9]:
print(query_labels)

['p', 'e', 'p', 'p', 'p', 'e', 'e', 'e', 'p', 'e']


In [10]:
wb_training = Workbook()

In [11]:
path = 'D:\EDUCATION\Data Mining\\training_database.xlsx'

In [12]:
wb_training = load_workbook(path)

In [13]:
ws_training = wb_training.active

In [14]:
training_tuple_cells = ws_training['A']
#print(tuple_cells)

In [15]:
training_labels = []
for cell_num in range(1, len(training_tuple_cells)+1):
    current_cell = 'A'+str(cell_num)
    training_labels.append(ws_training[current_cell].value)

num_training_data = len(training_labels)
print('Total no of data =', num_training_data)
#print('training_labels =', training_labels)

Total no of data = 8115


In [16]:
num_features = len(list(ws_training.rows)[0]) - 1
print('Total no. of features =', num_features)

Total no. of features = 22


In [17]:
query_numpy_array = np.zeros(shape=(num_query_data, num_features+1), dtype=np.float32)
query_data = list(ws_query.rows)

print(len(query_numpy_array[0]))
print(len(query_data[0]))

23
23


In [18]:
for i in range(len(query_data)):
    #print('i=',i)
    for j in range(len(query_data[i])):
        #print('j=',j)
        #print('f=',float(ord(query_data[i][j].value)))
        query_numpy_array[i][j] = float(ord(query_data[i][j].value))
        #print('q=',query_numpy_array[i][j],'\n')

query_numpy_array = np.delete(query_numpy_array, 0, axis=1) # deleting labels from reature matrix (0-th col of every row)
print(query_numpy_array.shape)

(10, 22)


In [19]:
training_numpy_array = np.zeros(shape=(num_training_data, num_features+1), dtype=np.float32)
training_data = list(ws_training.rows)

In [20]:
for i in range(len(training_data)):
    for j in range(len(training_data[i])):
        training_numpy_array[i][j] = float(ord(training_data[i][j].value))
        
training_numpy_array = np.delete(training_numpy_array, 0, axis=1) # deleting labels from reature matrix (0-th col of every row)
print(training_numpy_array.shape)

(8115, 22)


In [21]:
#start = timer()
total_correct_classification = 0
for query_data in range(num_query_data): #range(4,5): #
    true_label = query_labels[query_data]
    proximity_list = []
    max_current_similarity = 0
    max_features_matched_for = -1 #index of training data
    
    print('Testing for query data',query_data,'[original class -',true_label,']','...')
    for training_data in range(num_training_data): #range(5000) #range(5000,7001):#
        
        current_similarity = 0
        for query_feature,training_feature in zip(query_numpy_array[query_data],training_numpy_array[training_data]):
            if query_feature==training_feature:
                current_similarity += 1
        
        if max_current_similarity < current_similarity:
            max_current_similarity = current_similarity
            max_features_matched_for = training_data
        
        proximity = (num_features - current_similarity) / num_features
        proximity_list.append(proximity)

    derived_label = ''
    for index in range(len(proximity_list)):
        if proximity_list[index]==min(proximity_list):
            derived_label = training_labels[index]  
    print('\n    Class - '+'\033[94m'+'\033[1m'+str(derived_label)+'\033[0m'+' with Proximity = '+str(min(proximity_list)))

    if true_label==derived_label:
        total_correct_classification += 1
        print('    Classification is '+'\033[92m'+'\033[1m'+'correct!'+'\033[0m')
    else:
        print('    Classification is '+'\033[91m'+'\033[1m'+'wrong!'+'\033[0m')
    
    if max_features_matched_for > -1:
        print('    Maximum similarity were found in',max_current_similarity,\
              'feature(s) for training data',max_features_matched_for,'.\n')
        print('    query=\n', end='    ')
        [print(chr(i),', ',end = '') for i in query_numpy_array[query_data]]
        print('\n    training =\n', end='    ')
        [print(chr(i),', ',end = '') for i in training_numpy_array[max_features_matched_for]]
        print('\n')
        
    print('    -----------------------------------\n')
    
print(total_correct_classification,' out of',num_query_data,'are classified correctly!')
print('Classification accuracy = '+'\033[94m'+'\033[1m'+str((total_correct_classification/num_query_data)*100)+'%'+'\033[0m')

duration = timer() - start
print('Time elasped =',duration,'sec(s)')
    

Testing for query data 0 [original class - p ] ...

    Class - p with Proximity = 0.0
    Classification is correct!
    Maximum similarity were found in 22 feature(s) for training data 8114 .

    query=
    f , y , c , f , m , a , c , b , y , e , c , k , y , c , c , p , w , n , n , w , c , d , 
    training =
    f , y , c , f , m , a , c , b , y , e , c , k , y , c , c , p , w , n , n , w , c , d , 

    -----------------------------------

Testing for query data 1 [original class - e ] ...

    Class - e with Proximity = 0.045454545454545456
    Classification is correct!
    Maximum similarity were found in 21 feature(s) for training data 6999 .

    query=
    x , s , n , f , n , a , c , b , y , e , ? , s , s , o , o , p , o , o , p , o , v , l , 
    training =
    k , s , n , f , n , a , c , b , y , e , ? , s , s , o , o , p , o , o , p , o , v , l , 

    -----------------------------------

Testing for query data 2 [original class - p ] ...

    Class - p with Proximity = 0.

 Some notes:

In [22]:
print('sth' + '\033[1m' + ' Hello ' + '\033[0m' + 'sth')
print('sth' + '\033[94m' + ' Hello ' + '\033[0m' + 'sth')
print('sth' + '\033[94m' + '\033[1m' + ' Hello ' + '\033[0m'+'sth')

sth Hello sth
sth Hello sth
sth Hello sth


In [23]:
#Time elasped = 489.1767821029779 #without @jit
#Time elasped = 483.1280999945735 #with @jit
#Time elasped = 0.008947741956710208 #with numpy
#Time elasped = 0.007173661224927543 #with numpy and zip

In [24]:
import numpy as np
l = [1,2,3,4,5]
print(type(l))
l1 = np.asarray(l)
print(type(l1))

<class 'list'>
<class 'numpy.ndarray'>


In [25]:
type(np.asarray(list(ws_query.rows))[0][0].value)

str

In [26]:
type(ord('p'))

int

In [27]:
x=[1,2,3,4]
y=[4,3,2,1]
for i,j in zip(x,x):
    print(i,', ',j)

1 ,  1
2 ,  2
3 ,  3
4 ,  4


In [28]:
l = np.zeros(shape=(10,3), dtype=np.float32)
print(l)
print([0])
print(l[0][0])

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[0]
0.0


In [29]:
l = len(training_numpy_array)
print(l)
print(training_numpy_array[0])
training_numpy_array.shape

8115
[ 120.  115.  110.  116.  112.  102.   99.  110.  107.  101.  101.  115.
  115.  119.  119.  112.  119.  111.  112.  107.  115.  117.]


(8115, 22)

In [30]:
a = [0,1,2,3,4]
print(len(a))
for i in range(1, len(a)):
    print('i=',i,'; a=',a[i])

5
i= 1 ; a= 1
i= 2 ; a= 2
i= 3 ; a= 3
i= 4 ; a= 4


In [31]:
arr = np.array([[1,2,3,4,5],
                [6,7,8,9,0],
                [11,22,33,44,55]])
print(arr)
print(arr.shape)
arr = np.delete(arr, 0, axis=1)
print(arr)
print(arr.shape)

[[ 1  2  3  4  5]
 [ 6  7  8  9  0]
 [11 22 33 44 55]]
(3, 5)
[[ 2  3  4  5]
 [ 7  8  9  0]
 [22 33 44 55]]
(3, 4)
